# Implementing Round-Robin Communication with AutoGen - SOLUTION

> **Learning Objectives:**
> - Understand the basic concepts of round-robin communication
> - Begin exploring the AutoGen framework
> - Observe multiple agents communicating with each other
> - Implement composite termination conditions

## Introduction

In this lab, we will implement a round-robin communication using the AutoGen framework. This is a basic form of Agentic AI and is a simple way to get started with the AutoGen framework.

In round-robin, agents take turns to communicate with each other. This order is sequential and is repeated until a termination condition is met. For example, we might have agents: A, B, and C. The order of communication would be A -> B -> C -> A -> B -> C -> ... and so on. Once a termination condition is met, the communication stops.

### AutoGen Framework
[AutoGen](https://microsoft.github.io/autogen/stable/index.html) is "a framework for building AI agents and applications" that is developed by Microsoft. It is designed to be a flexible and extensible framework that can be used to build a wide range of AI applications.

We will be working with the `autogen-agentchat` component which is design for building conversational single and multi-agent systems. This component is built on top of the `autogen` framework and provides a simple way to build agents that can communicate with each other.

## The Scenario
Our agents will be tasked with writing a social media post for a fictional product named "FlowState". At the end of the lab, we will have three agents that will write, critique, and approve the social media post. Our agents are:

1. **Writer:** This agent will write the social media post. To ensure that we have a good example of communication, we will use the system prompt to force the writer to write poor quality posts.
2. **Writing Critic:** This agent will critique the post written by the writer. It will provide feedback on the quality of the post.
3. **Brand Manager:** This agent will ensure that the post meets the brand guidelines. If the post meets the guidelines, the post will be approved. If not, the writer will be asked to revise the post.


## Getting Started
Let's start by installing the required libraries and setting the OpenAI API key. The OpenAI API key is required to access the OpenAI models.

Run the following cells to install the required libraries and set the OpenAI API key.


In [2]:
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU openai==1.61.1 autogen-agentchat==0.4.6 autogen-ext[openai]

import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Enter your OpenAI API key:  ········


Next, we will define our product. The product is named "FlowState" and is a fictional smartwatch that helps users to improve their focus and productivity. We will use this product as the basis for our social media post.

In [4]:
PRODUCT_DESCRIPTION = """
**Product Description:**
FlowState is a smartwatch designed to help people focus.

**Features:**
    * Focus modes: Silences notifications so it's easier to perform deep work.
    * Smart notification filtering: Uses AI to prioritize important messages and alerts.
    * Task management integration: Connects with apps like Todoist and Trello to make it easier to manage tasks.
    * Long-lasting battery: It runs for several days.
""".strip()

## Core

### Step 1 - Write a Single Agent

To begin, we will write a round-robin chat with a single agent. This will help us understand the basic concepts of the AutoGen framework. The first agent will be the writer.

We need to setup a few things before we can start writing the agent. In the next cell, we will create the model client that we will use to interact with the OpenAI API. Run the following cell to create the model client.

In [5]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

Next, we will define our termination condition. We will use the key-phrase "SUBMIT POST" as the termination condition. The agent will be instructed to use this key-phrase to submit the post. Once the key-phrase is detected, the communication will stop.

Run the following cell to define the termination condition.

In [6]:
from autogen_agentchat.conditions import TextMentionTermination

termination = TextMentionTermination("SUBMIT POST")

We're ready to create the writer agent! In the next cell, we will define the writer agent. The agent will be defined by a system prompt that instructs the agent on its role and the termination condition.

Each agent needs a name. This name must be a valid Python variable name. We will use the name `writer` for the writer agent.

Feel free to modify the system prompt to customize the agent's behavior. Once you are ready, run the following cell to define the writer agent.

In [7]:
from autogen_agentchat.agents import AssistantAgent
import textwrap

writer = AssistantAgent(
    name="writer",
    system_message=textwrap.dedent("""
        Your are a Junior marketing specialist.
        You write and edit social media posts.
        You are still learning. Your first drafts are long and lack readability. Your first drafts
        also overuse jargon, buzzwords, and emoji. Some of the content is off-topic or irrelevant.

        Output only post content.

        Reply only "SUBMIT POST" if the previous post is ready to be submitted.
        Do not submit posts until they have been reviewed and revised.
    """.strip()),
    model_client=model_client,
)

With these core components in place, we can now setup and invoke the round-robin chat. In the next cell, we will create the chat and run it with the writer agent.

We'll have the agent write a social media post relevant to the '#DigitalMinimalism' trend. Run the following cell to watch the writer agent in action.

In [8]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console


# The group chat will only have the writer.
group_chat = RoundRobinGroupChat([writer], termination_condition=termination, max_turns=12)

# `run_stream` returns an async generator to stream the intermediate messages.
stream = group_chat.run_stream(task=textwrap.dedent(f"""
    {PRODUCT_DESCRIPTION}

    Write a social media post to promote this product related to the following social media trend:

    Trend: #DigitalMinimalism
    Description: Intentional reduction of tech use to reclaim time and attention.
    """.strip()))

result = None

# Stream the messages until we get the result.
await Console(stream)

---------- user ----------
**Product Description:**
FlowState is a smartwatch designed to help people focus.

**Features:**
    * Focus modes: Silences notifications so it's easier to perform deep work.
    * Smart notification filtering: Uses AI to prioritize important messages and alerts.
    * Task management integration: Connects with apps like Todoist and Trello to make it easier to manage tasks.
    * Long-lasting battery: It runs for several days.

    Write a social media post to promote this product related to the following social media trend:

    Trend: #DigitalMinimalism
    Description: Intentional reduction of tech use to reclaim time and attention.
---------- writer ----------
Unlock your full potential with FlowState, the smartwatch designed specifically for those who want to embrace #DigitalMinimalism! 🌿✨ 

In a world full of distractions, FlowState helps you reclaim your focus and attention. Here’s how: 

🔕 **Focus Modes**: Silence unwanted notifications and dive deep

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="**Product Description:**\nFlowState is a smartwatch designed to help people focus.\n\n**Features:**\n    * Focus modes: Silences notifications so it's easier to perform deep work.\n    * Smart notification filtering: Uses AI to prioritize important messages and alerts.\n    * Task management integration: Connects with apps like Todoist and Trello to make it easier to manage tasks.\n    * Long-lasting battery: It runs for several days.\n\n    Write a social media post to promote this product related to the following social media trend:\n\n    Trend: #DigitalMinimalism\n    Description: Intentional reduction of tech use to reclaim time and attention.", type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=234, completion_tokens=191), content='Unlock your full potential with FlowState, the smartwatch designed specifically for those who want to embrace #DigitalMinimalism! 🌿✨ \n\n

Not the best post, right? The writer agent is intentionally writing a poor quality post to demonstrate the communication process. It likely didn't review the post before submitting it and probably made the post too long.

Even though it has the opportunity to revise the post, the writer agent is not doing a great job. This is where the writing critic agent comes in!

### Step 2 - Write the Writing Critic Agent

In this step, we will write the writing critic agent. This agent will critique the post written by the writer agent. The writing critic agent will provide feedback on the quality of the post.


In the cell below, write the following:

1. A new writer agent that doesn't have the termination condition.
    * You can copy the writer agent code from the previous cell and remove the termination condition from the prompt.
2. A new agent for the writing critic.
    * Here is a sample prompt:
        ```
        You are a Expert social media marketing reviewer.

        Good posts are:
            - Under 100 words
            - Use new lines and bullet points
            - Avoid jargon and buzzwords
            - Include a call to action
            - Are relevant to the trend
            - Use emojis sparingly
        
        Provide specific, actionable feedback.
        Do not rewrite the post.

        Reply only "WRITING APPROVED", if there is no feedback and the post is ready to be submitted. 
        ```

Once you have written the code, run the cell and observe the communication between the writer and the writing critic! Iterate as much as you need to understand the communication process.



In [9]:
# [SOLUTION]
writer = AssistantAgent(
    name="writer",
    system_message=textwrap.dedent("""
        Your are a Junior marketing specialist.
        You write and edit social media posts.
        You are still learning. Your first drafts are long and lack readability. Your first drafts
        also overuse jargon, buzzwords, and emoji. Some of the content is off-topic or irrelevant.

        Output only post content.
    """.strip()),
    model_client=model_client,
)

critic = AssistantAgent(
    name="writing_critic",
    description="Expert social media marketing reviewer.",
    system_message=textwrap.dedent("""
        You are a Expert social media marketing reviewer.

        Good posts are:
            - Under 100 words
            - Use new lines and bullet points
            - Avoid jargon and buzzwords
            - Include a call to action
            - Are relevant to the trend
            - Use emojis sparingly
        
        Provide specific, actionable feedback.
        Do not rewrite the post.

        Reply only "WRITING APPROVED", if there is no feedback and the post is ready to be submitted.
    """.strip()),
    model_client=model_client,
)

termination = TextMentionTermination("WRITING APPROVED")

group_chat = RoundRobinGroupChat([writer, critic], termination_condition=termination, max_turns=12)

stream = group_chat.run_stream(task=textwrap.dedent(f"""
    {PRODUCT_DESCRIPTION}

    Write a social media post to promote this product related to the following social media trend:

    Trend: #DigitalMinimalism
    Description: Intentional reduction of tech use to reclaim time and attention.
    """.strip()))

await Console(stream)
# [/SOLUTION]

# writer = None

# critic = None

# termination = TextMentionTermination("WRITING APPROVED")

# group_chat = RoundRobinGroupChat([writer, critic], termination_condition=termination, max_turns=12)

# stream = group_chat.run_stream(task=textwrap.dedent(f"""
#     {PRODUCT_DESCRIPTION}

#     Write a social media post to promote this product related to the following social media trend:

#     Trend: #DigitalMinimalism
#     Description: Intentional reduction of tech use to reclaim time and attention.
#     """.strip()))

# await Console(stream)

---------- user ----------
**Product Description:**
FlowState is a smartwatch designed to help people focus.

**Features:**
    * Focus modes: Silences notifications so it's easier to perform deep work.
    * Smart notification filtering: Uses AI to prioritize important messages and alerts.
    * Task management integration: Connects with apps like Todoist and Trello to make it easier to manage tasks.
    * Long-lasting battery: It runs for several days.

    Write a social media post to promote this product related to the following social media trend:

    Trend: #DigitalMinimalism
    Description: Intentional reduction of tech use to reclaim time and attention.
---------- writer ----------
✨ Embrace #DigitalMinimalism with FlowState! ✨ 

In a world full of distractions, our smartwatch is your perfect companion for deep work and focus. 

🛑 Silence unnecessary notifications with our Focus Modes, ensuring that only the most important alerts come through. 

🔍 Smart notification filtering

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="**Product Description:**\nFlowState is a smartwatch designed to help people focus.\n\n**Features:**\n    * Focus modes: Silences notifications so it's easier to perform deep work.\n    * Smart notification filtering: Uses AI to prioritize important messages and alerts.\n    * Task management integration: Connects with apps like Todoist and Trello to make it easier to manage tasks.\n    * Long-lasting battery: It runs for several days.\n\n    Write a social media post to promote this product related to the following social media trend:\n\n    Trend: #DigitalMinimalism\n    Description: Intentional reduction of tech use to reclaim time and attention.", type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=203, completion_tokens=142), content='✨ Embrace #DigitalMinimalism with FlowState! ✨ \n\nIn a world full of distractions, our smartwatch is your perfect companion for deep wor

Way to go! The writing critic agent is providing feedback on the quality of the post. The writer agent can use this feedback to improve the post. However, the writer agent is not the only one who needs to approve the post. The brand manager agent will also need to approve the post.

### Step 3 - Write the Brand Manager Agent

The brand manager agent will ensure that the post meets the brand guidelines. If the post meets the guidelines, the post will be approved. If not, the writer will be asked to revise the post and the review process will start again.

Here is a sample prompt for the brand manager agent:

```
You are an experienced branding specialist.
FlowState's brand voice is sophisticated, modern, and focused.
FlowState doesn't use slang or emojis.
Posts should evoke a sense of calm and productivity.
Provide specific, actionable feedback on whether the post is on-brand.
Reply only "BRANDING APPROVED" if the post is on-brand, high-quality content.
```

In the cell below, write the following code:

1. A new brand manager agent.
    * You can copy the sample prompt above or write your own prompt.
2. A new round-robin chat that includes the writer, writing critic, and brand manager agents.
3. A new termination condition for the chat.
    * We want both the writing critic and the brand manager to approve the post before it is submitted.
    * To do this, we can use the `&` (and-operator) to combine two conditions.
    * e.g., `TextMentionTermination("Keyword 1") & TextMentionTermination("Keyword 2")` will terminate the chat when both "Keyword 1" and "Keyword 2" are mentioned.

In [10]:
# [SOLUTION]
branding = AssistantAgent(
    name="brand_manager",
    system_message=textwrap.dedent("""
        You are an experienced branding specialist.
        FlowState's brand voice is sophisticated, modern, and focused.
        FlowState doesn't use slang or emojis.
        Posts should evoke a sense of calm and productivity.
        Provide specific, actionable feedback on whether the post is on-brand.
        Reply only "BRANDING APPROVED" if the post is on-brand, high-quality content.
    """),
    model_client=model_client,
)

# The group chat will alternate between the writer and the critic.
group_chat = RoundRobinGroupChat(
    [writer, critic, branding ],
    termination_condition=TextMentionTermination("WRITING APPROVED") & TextMentionTermination("BRANDING APPROVED"),
    max_turns=12
)

# `run_stream` returns an async generator to stream the intermediate messages.
stream = group_chat.run_stream(task=textwrap.dedent(f"""
    {PRODUCT_DESCRIPTION}

    Write a social media post to promote this product related to the following social media trend:

    Trend: #DigitalMinimalism
    Description: Intentional reduction of tech use to reclaim time and attention.
    """))

await Console(stream)

---------- user ----------

    **Product Description:**
FlowState is a smartwatch designed to help people focus.

**Features:**
    * Focus modes: Silences notifications so it's easier to perform deep work.
    * Smart notification filtering: Uses AI to prioritize important messages and alerts.
    * Task management integration: Connects with apps like Todoist and Trello to make it easier to manage tasks.
    * Long-lasting battery: It runs for several days.

    Write a social media post to promote this product related to the following social media trend:

    Trend: #DigitalMinimalism
    Description: Intentional reduction of tech use to reclaim time and attention.

---------- writer ----------
Stay focused with FlowState! ⌚️✨ 

In today’s tech-driven world, it's time to embrace #DigitalMinimalism. Our smartwatch helps you intentionally reduce distractions by:

- 🛑 Silencing notifications for deep work.
- 🔍 Prioritizing only the important messages with smart filtering.
- 📅 Integrati

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="\n    **Product Description:**\nFlowState is a smartwatch designed to help people focus.\n\n**Features:**\n    * Focus modes: Silences notifications so it's easier to perform deep work.\n    * Smart notification filtering: Uses AI to prioritize important messages and alerts.\n    * Task management integration: Connects with apps like Todoist and Trello to make it easier to manage tasks.\n    * Long-lasting battery: It runs for several days.\n\n    Write a social media post to promote this product related to the following social media trend:\n\n    Trend: #DigitalMinimalism\n    Description: Intentional reduction of tech use to reclaim time and attention.\n", type='TextMessage'), TextMessage(source='writer', models_usage=RequestUsage(prompt_tokens=737, completion_tokens=129), content="Stay focused with FlowState! ⌚️✨ \n\nIn today’s tech-driven world, it's time to embrace #DigitalMinimalism. Our smartwatch helps 

Great work! The brand manager agent is ensuring that the post meets the brand guidelines. The writer agent can use the feedback from the writing critic and the brand manager to improve the post. Once the post is approved by both the writing critic and the brand manager, the post can be submitted.

This is a simple example of round-robin communication using the AutoGen framework. You can continue to iterate on this example by adding more agents, changing the prompts, or modifying the termination conditions.

## Bonus Challenge 1

Round-robin communication can be used in a variety of scenarios. In this challenge, you will create a round-robin chat that write source code for based on the user's input. The chat will include the following agents:

1. **Developer:** This agent will write the source code.
2. **Code Reviewer:** This agent will review the source code and provide feedback.

In the cell below, write the necessary code to create the Developer and Code Reviewer agents.

<div class="alert alert-block alert-warning">

**Warning!** Generative AI can write code that is vulnerable, buggy, and inefficient! Use with caution and always review the generated code before running it.
</div>


In [ ]:
# YOUR CODE HERE